# AIC で説明変数の重要度を算出する

---

## 理論を理解する

ある説明変数 $x$ の重要度を求めるために、<br>
｢説明変数 $x$ が与えられたとき目的変数 $Y$ が $y$ となる確率｣ が (学習データから求めた) 反応率で表されるモデルを考え、<br>
そのモデルの AIC を説明変数 $x$ の重要度とすることを考える。

｢説明変数 $x$ が与えられたとき目的変数 $Y$ が $y$ となる確率｣ を次のようにモデル化する。<br>
ここで、$\#$ は集合の要素数、$i$ は学習データに含まれる観測の添え字、<br>
$x_i$ は観測 $i$ の説明変数の実現値、$y_i$ は観測 $i$ の目的変数の実現値を表す。

$$
P \left( Y = y \, ; \, x \right) =
  \frac
    { \# \left\{ i \mid x_i = x \land y_i = y \right\} }
    { \# \left\{ i \, \mid \, x_i = x \right\} }
$$

このモデルのパラメータ数 $k$ は、次のようになる。<br>
説明変数や目的変数の水準数は固定なので、自由度が1下がることに注意する。<br>
計算にあたり、目的変数が ｢反応する｣ ｢反応しない｣ の2水準であること ($\# \left\{ y \right\} = 2$) を用いた。

$$
k = \left( \# \left\{ x \right\} - 1 \right) \left( \# \left\{ y \right\} - 1 \right) = \# \left\{ x \right\} - 1 ~~ \left( \because \# \left\{ y \right\} = 2 \right)
$$

また、学習データに対する尤度 $L$ は次のようになる。<br>

$$
\begin{eqnarray*}
L &=& \prod_{ i } { P \left( Y = y_i \, ; \, x_i \right) } \\
&=& \prod_{ x } { \prod_{ y } { { P \left( Y = y \, ; \, x \right) }^{ \# \left\{ i \mid x_i = x \land y_i = y \right\} } } }
\end{eqnarray*}
$$

以上から、AIC は次のように計算できる。

$$
\begin{eqnarray*}
\text{AIC} &=& -2 \log{ L } + 2k \\
&=& -2 \log { \left( \prod_{ x } { \prod_{ y } { { P \left( Y = y \, ; \, x \right) }^{ \# \left\{ i \mid x_i = x \land y_i = y \right\} } } } \right) } +
  2 \left( \# \left\{ x \right\} - 1 \right) \\
&=& \sum_{ x } { \sum_{ y }
  { \left( -2 \bullet { \# \left\{ i \mid x_i = x \land y_i = y \right\} } \bullet
  \log { P \left( Y = y \, ; \, x \right) } \right) } } +
  2 \left( \# \left\{ x \right\} - 1 \right)
\end{eqnarray*}
$$

---

## 実装してみる

### 必要なパッケージを読み込む

In [1]:
library(tidyverse)

── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



### 反応率を計算する

In [2]:
calc_prob <-
  function(data) {
    data %>%
    # 水準別に件数をカウントする
    group_by(level) %>%
    mutate(n.level = n()) %>%
    ungroup() %>%
    # 水準別反応別に件数をカウントする
    count(level, y, n.level) %>%
    # 反応率と非反応率を計算する
    mutate(p = n / n.level)
  }

### AIC を計算する

In [3]:
calc_aic <-
  function(data) {
    data %>%
    mutate(aic = -2 * n * log(p)) %>%
    summarise(aic = sum(aic) + 2 * (length(unique(level)) - 1))
  }

---

## 実際に指標を計算してみる

### データを読み込む

サンプルデータとして、[SIGNATE](https://signate.jp/)の練習用コンペ ｢[【練習問題】銀行の顧客ターゲティング](https://signate.jp/competitions/1)｣ の学習データを読み込む。

In [4]:
train <- read.csv("bank/train.csv")
head(train, n = 10)

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<int>
1,1,39,blue-collar,married,secondary,no,1756,yes,no,cellular,3,apr,939,1,-1,0,unknown,1
2,2,51,entrepreneur,married,primary,no,1443,no,no,cellular,18,feb,172,10,-1,0,unknown,1
3,3,36,management,single,tertiary,no,436,no,no,cellular,13,apr,567,1,595,2,failure,1
4,4,63,retired,married,secondary,no,474,no,no,cellular,25,jan,423,1,-1,0,unknown,1
5,5,31,management,single,tertiary,no,354,no,no,cellular,30,apr,502,1,9,2,success,1
6,6,29,blue-collar,single,secondary,no,260,yes,no,unknown,2,jun,707,14,-1,0,unknown,1
7,7,37,services,married,secondary,no,52,yes,no,cellular,6,sep,908,1,185,9,success,1
8,8,32,technician,single,secondary,no,230,yes,no,cellular,18,may,442,1,266,8,failure,1
9,9,31,admin.,single,secondary,no,0,yes,no,cellular,7,may,895,2,295,2,failure,1


### 連続変数を離散化する

In [5]:
train <-
  train %>%
  mutate(duration = cut(duration, breaks = c(-Inf, 80, 127, 181, 260, 415, Inf))) %>%
  mutate(pdays = cut(pdays, breaks = c(-Inf, -1, 91, Inf))) %>%
  mutate(previous = cut(previous, breaks = c(-Inf, 0, Inf))) %>%
  mutate(balance = cut(balance, breaks = c(-Inf, 2, 175, 449, 953, 2281, Inf))) %>%
  mutate(campaign = cut(campaign, breaks = c(-Inf, 1, 2, 4, Inf))) %>%
  mutate(age = cut(age, breaks = c(-Inf, 30, 34, 39, 45, 52, 95))) %>%
  mutate(day = cut(day, breaks = c(-Inf, 6, 11, 15, 19, 25, Inf))) %>%
  mutate(id = cut(id, breaks = c(-Inf, 4521, 9042, 13564, 18085, 22607, Inf)))
train %>% sample_n(size = 10)

id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>
"(2.26e+04, Inf]","(45,52]",technician,married,secondary,no,"(953,2.28e+03]",yes,no,unknown,"(11,15]",may,"(127,181]","(-Inf,1]","(-Inf,-1]","(-Inf,0]",unknown,0
"(1.81e+04,2.26e+04]","(52,95]",management,married,primary,no,"(2.28e+03, Inf]",yes,no,cellular,"(11,15]",may,"(260,415]","(-Inf,1]","(-Inf,-1]","(-Inf,0]",unknown,0
"(1.36e+04,1.81e+04]","(39,45]",services,married,tertiary,no,"(2.28e+03, Inf]",no,no,cellular,"(19,25]",dec,"(127,181]","(1,2]","(91, Inf]","(0, Inf]",success,1
"(1.36e+04,1.81e+04]","(39,45]",technician,married,tertiary,no,"(-Inf,2]",yes,no,cellular,"(-Inf,6]",may,"(-Inf,80]","(-Inf,1]","(-Inf,-1]","(-Inf,0]",unknown,0
"(1.81e+04,2.26e+04]","(-Inf,30]",student,married,secondary,no,"(449,953]",yes,yes,cellular,"(11,15]",may,"(127,181]","(2,4]","(-Inf,-1]","(-Inf,0]",unknown,0
"(-Inf,4.52e+03]","(-Inf,30]",student,single,unknown,no,"(449,953]",no,no,cellular,"(25, Inf]",oct,"(-Inf,80]","(-Inf,1]","(-Inf,-1]","(-Inf,0]",unknown,0
"(4.52e+03,9.04e+03]","(39,45]",blue-collar,single,primary,no,"(953,2.28e+03]",yes,no,telephone,"(11,15]",may,"(-Inf,80]","(4, Inf]","(-Inf,-1]","(-Inf,0]",unknown,0
"(1.81e+04,2.26e+04]","(52,95]",management,married,tertiary,no,"(2.28e+03, Inf]",no,no,telephone,"(15,19]",apr,"(415, Inf]","(4, Inf]","(-Inf,-1]","(-Inf,0]",unknown,0
"(4.52e+03,9.04e+03]","(39,45]",management,divorced,tertiary,no,"(449,953]",no,yes,cellular,"(19,25]",jul,"(181,260]","(2,4]","(-Inf,-1]","(-Inf,0]",unknown,0


### 各説明変数に対する反応率を計算する

In [6]:
prob <-
  # 説明変数別に、該当する説明変数と目的変数の2変数からなるデータフレームを作成する
  tibble(variable = setdiff(colnames(train), "y")) %>%
  mutate(data = map(variable, ~ tibble(level = train[[.]], y = train$y))) %>%
  # 説明変数別に、反応率を計算する
  mutate(data = map(data, calc_prob))

In [7]:
# データフレームの中にデータフレームを含む構造になっている
prob

variable,data
<chr>,<list>
id,"1.0000000, 1.0000000, 2.0000000, 2.0000000, 3.0000000, 3.0000000, 4.0000000, 4.0000000, 5.0000000, 5.0000000, 6.0000000, 6.0000000, 0.0000000, 1.0000000, 0.0000000, 1.0000000, 0.0000000, 1.0000000, 0.0000000, 1.0000000, 0.0000000, 1.0000000, 0.0000000, 1.0000000, 4521.0000000, 4521.0000000, 4521.0000000, 4521.0000000, 4522.0000000, 4522.0000000, 4521.0000000, 4521.0000000, 4522.0000000, 4522.0000000, 4521.0000000, 4521.0000000, 4015.0000000, 506.0000000, 3988.0000000, 533.0000000, 3954.0000000, 568.0000000, 3967.0000000, 554.0000000, 4012.0000000, 510.0000000, 4018.0000000, 503.0000000, 0.8880779, 0.1119221, 0.8821057, 0.1178943, 0.8743919, 0.1256081, 0.8774607, 0.1225393, 0.8872180, 0.1127820, 0.8887414, 0.1112586"
age,"1.000000e+00, 1.000000e+00, 2.000000e+00, 2.000000e+00, 3.000000e+00, 3.000000e+00, 4.000000e+00, 4.000000e+00, 5.000000e+00, 5.000000e+00, 6.000000e+00, 6.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 4.201000e+03, 4.201000e+03, 4.777000e+03, 4.777000e+03, 5.012000e+03, 5.012000e+03, 4.439000e+03, 4.439000e+03, 4.230000e+03, 4.230000e+03, 4.469000e+03, 4.469000e+03, 3.525000e+03, 6.760000e+02, 4.254000e+03, 5.230000e+02, 4.501000e+03, 5.110000e+02, 4.036000e+03, 4.030000e+02, 3.859000e+03, 3.710000e+02, 3.779000e+03, 6.900000e+02, 8.390859e-01, 1.609141e-01, 8.905171e-01, 1.094829e-01, 8.980447e-01, 1.019553e-01, 9.092138e-01, 9.078621e-02, 9.122931e-01, 8.770686e-02, 8.456030e-01, 1.543970e-01"
job,"1.000000e+00, 1.000000e+00, 2.000000e+00, 2.000000e+00, 3.000000e+00, 3.000000e+00, 4.000000e+00, 4.000000e+00, 5.000000e+00, 5.000000e+00, 6.000000e+00, 6.000000e+00, 7.000000e+00, 7.000000e+00, 8.000000e+00, 8.000000e+00, 9.000000e+00, 9.000000e+00, 1.000000e+01, 1.000000e+01, 1.100000e+01, 1.100000e+01, 1.200000e+01, 1.200000e+01, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 3.085000e+03, 3.085000e+03, 5.886000e+03, 5.886000e+03, 9.140000e+02, 9.140000e+02, 7.650000e+02, 7.650000e+02, 5.620000e+03, 5.620000e+03, 1.391000e+03, 1.391000e+03, 9.450000e+02, 9.450000e+02, 2.506000e+03, 2.506000e+03, 5.570000e+02, 5.570000e+02, 4.491000e+03, 4.491000e+03, 7.900000e+02, 7.900000e+02, 1.780000e+02, 1.780000e+02, 2.700000e+03, 3.850000e+02, 5.456000e+03, 4.300000e+02, 8.340000e+02, 8.000000e+01, 6.970000e+02, 6.800000e+01, 4.853000e+03, 7.670000e+02, 1.080000e+03, 3.110000e+02, 8.280000e+02, 1.170000e+02, 2.282000e+03, 2.240000e+02, 3.910000e+02, 1.660000e+02, 4.013000e+03, 4.780000e+02, 6.600000e+02, 1.300000e+02, 1.600000e+02, 1.800000e+01, 8.752026e-01, 1.247974e-01, 9.269453e-01, 7.305471e-02, 9.124726e-01, 8.752735e-02, 9.111111e-01, 8.888889e-02, 8.635231e-01, 1.364769e-01, 7.764198e-01, 2.235802e-01, 8.761905e-01, 1.238095e-01, 9.106145e-01, 8.938547e-02, 7.019749e-01, 2.980251e-01, 8.935649e-01, 1.064351e-01, 8.354430e-01, 1.645570e-01, 8.988764e-01, 1.011236e-01"
marital,"1.000000e+00, 1.000000e+00, 2.000000e+00, 2.000000e+00, 3.000000e+00, 3.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 3.055000e+03, 3.055000e+03, 1.641100e+04, 1.641100e+04, 7.662000e+03, 7.662000e+03, 2.691000e+03, 3.640000e+02, 1.474400e+04, 1.667000e+03, 6.519000e+03, 1.143000e+03, 8.808511e-01, 1.191489e-01, 8.984218e-01, 1.015782e-01, 8.508222e-01, 1.491778e-01"
education,"1.000000e+00, 1.000000e+00, 2.000000e+00, 2.000000e+00, 3.000000e+00, 3.000000e+00, 4.000000e+00, 4.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00, 1.000000e+00, 4.150000e+03, 4.150000e+03, 1.388200e+04, 1.38820

In [8]:
# 試しに取り出してみると、確かにデータフレームが格納されていることが確認できる
prob[[1, "data"]]

level,y,n.level,n,p
<fct>,<int>,<int>,<int>,<dbl>
"(-Inf,4.52e+03]",0,4521,4015,0.8880779
"(-Inf,4.52e+03]",1,4521,506,0.1119221
"(4.52e+03,9.04e+03]",0,4521,3988,0.8821057
"(4.52e+03,9.04e+03]",1,4521,533,0.1178943
"(9.04e+03,1.36e+04]",0,4522,3954,0.8743919
"(9.04e+03,1.36e+04]",1,4522,568,0.1256081
"(1.36e+04,1.81e+04]",0,4521,3967,0.8774607
"(1.36e+04,1.81e+04]",1,4521,554,0.1225393
"(1.81e+04,2.26e+04]",0,4522,4012,0.8872180


### 全体に対する AIC を計算する

In [9]:
aic.all <-
  # 1水準の説明変数とみなして、説明変数と目的変数の2変数からなるデータフレームを作成する
  tibble(level = NA, y = train$y) %>%
  # 反応率を計算する
  calc_prob() %>%
  # AIC を計算する
  calc_aic() %>%
  # データフレームである必要がないので、値だけ取り出す
  .$aic
aic.all

[1] 19581.35

### 各説明変数に対する AIC を計算する

In [10]:
aic <-
  prob %>%
  # 説明変数別に、AIC を計算する
  mutate(data = map(data, calc_aic)) %>%
  # ネストしているデータフレームを展開し、フラットにする
  unnest(cols = data) %>%
  # 説明変数別の AIC と全体の AIC の差を計算する
  mutate(aic.all = aic.all) %>%
  mutate(aic.diff = aic - aic.all)
aic

variable,aic,aic.all,aic.diff
<chr>,<dbl>,<dbl>,<dbl>
id,19583.42,19581.35,2.069980
age,19380.60,19581.35,-200.749596
job,19144.88,19581.35,-436.469343
marital,19474.46,19581.35,-106.881587
education,19463.35,19581.35,-117.995573
default,19574.56,19581.35,-6.790677
balance,19323.78,19581.35,-257.562658
housing,19051.32,19581.35,-530.023353
loan,19451.86,19581.35,-129.486535


### 1枚の表にまとめる

In [11]:
output <-
  prob %>%
  # 見やすいよう水準でソートする (level を character に変換する前にソートする必要がある)
  mutate(data = map(data, ~ arrange(., level))) %>%
  # 次の unnest() で異なる factor を1列にまとめたいので、あらかじめ character に変換しておく
  mutate(data = map(data, ~ mutate(., level = as.character(level)))) %>%
  # ネストしているデータフレームを展開し、フラットにする
  unnest(cols = data) %>%
  # 各説明変数の AIC を結合する
  left_join(aic, by = "variable") %>%
  # 見やすいよう AIC の小さい順にソートする
  arrange(aic.diff) %>%
  # 不要な行と列を削除する
  filter(y == 1) %>%
  select(-y, -aic, -aic.all) %>%
  # 見やすいよう列名を日本語にする
  rename(`項目` = variable, `範囲` = level, `件数` = n.level, `反応件数` = n, `反応率` = p, AIC = aic.diff)
output

項目,範囲,件数,反応件数,反応率,AIC
<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
duration,"(-Inf,80]",4558,22,0.004826678,-3470.6917
duration,"(80,127]",4495,123,0.027363737,-3470.6917
duration,"(127,181]",4497,276,0.061374249,-3470.6917
duration,"(181,260]",4529,450,0.099359682,-3470.6917
duration,"(260,415]",4531,661,0.145883911,-3470.6917
duration,"(415, Inf]",4518,1642,0.363435148,-3470.6917
poutcome,failure,2969,391,0.131694173,-1549.1827
poutcome,other,1123,173,0.154051647,-1549.1827
poutcome,success,886,574,0.647855530,-1549.1827


---

## 動作環境

In [12]:
sessionInfo()

R version 3.5.2 (2018-12-20)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Debian GNU/Linux 10 (buster)

Matrix products: default
BLAS: /usr/lib/x86_64-linux-gnu/blas/libblas.so.3.8.0
LAPACK: /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3.8.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] forcats_0.5.1   stringr_1.4.0   dplyr_1.0.4     purrr_0.3.4    
[5] readr_1.4.0     tidyr_1.1.2     tibble_3.0.6    ggplot2_3.3.3  
[9] tidyverse_1.3.0

loaded via a namespace (and not attached):
 [1] pbdZMQ_0.3-5      tidyselect_1.1.0  repr_1.1.3        haven_2.3.1      
 [5] colorspace_2.0-0  vctrs_0.3.6       